In [1]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout
from keras_tqdm import TQDMNotebookCallback
import keras.callbacks

from sqlalchemy import create_engine
import json
import pickle

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
engine = create_engine('sqlite:///tab.db')
conn = engine.connect()
conn
dfo = pd.read_sql_table('race', conn)
print(f'{len(dfo)} races loaded!')
dfo = dfo.loc[dfo['race_type'].isin(['R'])]
print(f'{len(dfo)} horse races!')

7306 races loaded!
2426 horse races!


In [3]:
dfo['results'] = dfo['results_data'].map(json.loads)
dfo['runners'] = dfo['runners_data'].map(json.loads)
dfo.head()

,id,meeting_name,location,venue_mnemonic,race_type,meeting_date,race_number,race_name,race_start_time,race_status,race_distance,results_data,num_runners,runners_data,results,runners
1060,1610,RANDWICK,NSW,S,R,2017-09-16,1,TAB HIGHWAY HANDICAP,2017-09-16 02:15:00,Paying,1000,"[[4], [10], [2], [5]]",10.0,"[{""runnerName"": ""AZZIE'S READY"", ""runnerNumber...","[[4], [10], [2], [5]]","[{'runnerName': 'AZZIE'S READY', 'runnerNumber..."
1061,1611,RANDWICK,NSW,S,R,2017-09-16,2,SHOOT OUT MILE,2017-09-16 02:50:00,Paying,1600,"[[3], [5], [8], [7]]",8.0,"[{""runnerName"": ""GOLD HORIZON"", ""runnerNumber""...","[[3], [5], [8], [7]]","[{'runnerName': 'GOLD HORIZON', 'runnerNumber'..."
1062,1612,RANDWICK,NSW,S,R,2017-09-16,3,SCHWEPPES HERITAGE STAKES,2017-09-16 03:25:00,Paying,1100,"[[2], [1], [8], [6]]",8.0,"[{""runnerName"": ""LUCKY LOUIE"", ""runnerNumber"":...","[[2], [1], [8], [6]]","[{'runnerName': 'LUCKY LOUIE', 'runnerNumber':..."
1063,1613,RANDWICK,NSW,S,R,2017-09-16,4,BILL RITCHIE HANDICAP,2017-09-16 04:00:00,Paying,1400,"[[7], [3], [4], [6]]",7.0,"[{""runnerName"": ""PAJARO"", ""runnerNumber"": 6, ""...","[[7], [3], [4], [6]]","[{'runnerName': 'PAJARO', 'runnerNumber': 6, '..."
1064,1614,RANDWICK,NSW,S,R,2017-09-16,5,MCGRATH ESTATE TEA ROSE STAKES,2017-09-16 04:35:00,Paying,1400,"[[6], [2], [5], [1]]",10.0,"[{""runnerName"": ""PANDEMONIUM"", ""runnerNumber"":...","[[6], [2], [5], [1]]","[{'runnerName': 'PANDEMONIUM', 'runnerNumber':..."


In [5]:
# extract runners from races
data_all = pd.DataFrame()
for ri, race in dfo.iterrows():
#     print(race)

    # add num_runners to all runners
    for runner in race['runners']:
#         print(runner)
        runner['num_runners'] = race['num_runners']
        runner['race_type'] = race['race_type']

    data_all = data_all.append(race['runners'])
    
data_all.tail(10)

,barrierNumber,claimAmount,finishingPosition,fixedOdds,num_runners,odds_perc,odds_scale,odds_win,parimutuel,prediction,probability,race_type,rank_win,riderDriverName,runnerName,runnerNumber,trainerName
0,3,-1.0,0,"{'returnWin': 6, 'returnWinOpen': 7.5, 'return...",9.0,0.166667,0.139845,6.0,"{'bettingStatus': 'Normal', 'returnWin': 5.9, ...",0.120137,0.115325,R,2,S LEVEY,GALACTIC,9,R HANNON
1,7,-1.0,1,"{'returnWin': 7, 'returnWinOpen': 13, 'returnP...",9.0,0.142857,0.119867,7.0,"{'bettingStatus': 'Normal', 'returnWin': 8.6, ...",0.087129,0.083639,R,4,C BISHOP,EL BORRACHO,10,S DOW
2,8,-1.0,0,"{'returnWin': 6, 'returnWinOpen': 4.2, 'return...",9.0,0.166667,0.139845,6.0,"{'bettingStatus': 'Normal', 'returnWin': 3.7, ...",0.230551,0.221317,R,2,P DOBBS,STRATEGIC,3,R HANNON
3,6,2.5,0,"{'returnWin': 4.4, 'returnWinOpen': 5.5, 'retu...",9.0,0.227273,0.190698,4.4,"{'bettingStatus': 'Normal', 'returnWin': 4.5, ...",0.182203,0.174905,R,1,P PILLEY,SARDENYA,4,R CHARLTON
4,1,-1.0,2,"{'returnWin': 9.5, 'returnWinOpen': 5.5, 'retu...",9.0,0.105263,0.088323,9.5,"{'bettingStatus': 'Normal', 'returnWin': 16, '...",0.069569,0.066783,R,6,R TATE,MOTABASSIM,5,D W THOMPSON
5,5,-1.0,0,"{'returnWin': 17, 'returnWinOpen': 13, 'return...",9.0,0.058824,0.049357,17.0,"{'bettingStatus': 'Normal', 'returnWin': 27.4,...",0.054282,0.052108,R,9,D COSTELLO,GENERAL ZOFF,6,W R MUIR
6,7,-1.0,0,"{'returnWin': 0, 'returnWinOpen': 0, 'returnPl...",9.0,0.000000,0.000000,0.0,"{'bettingStatus': 'Scratched', 'returnWin': 0,...",0.000000,0.000000,R,10,F NORTON,HEMINGFORD,7,CHARLIE FELLOWES
7,2,1.5,3,"{'returnWin': 7.5, 'returnWinOpen': 7.5, 'retu...",9.0,0.133333,0.111876,7.5,"{'bettingStatus': 'Normal', 'returnWin': 7.8, ...",0.132872,0.127550,R,5,E GREATREX,DEADLY REEL,8,ARCHIE WATSON
8,9,-1.0,0,"{'returnWin': 10, 'returnWinOpen': 7.5, 'retur...",9.0,0.100000,0.083907,10.0,"{'bettingStatus': 'Normal', 'returnWin': 9.6, ...",0.083980,0.080616,R,7,R FFRENCH,RED FORCE ONE,1,TOM DASCOMBE
9,4,-1.0,0,"{'returnWin': 11, 'returnWinOpen': 11, 'return...",9.0,0.090909,0.076279,11.0,"{'bettingStatus': 'Normal', 'returnWin': 14, '...",0.081001,0.077757,R,8,S DROWNE,DADDIES GIRL,2,B R MILLMAN


In [6]:
# drop scratched
data = data_all.dropna(subset=['odds_win'])
data = data[(data.odds_win > 0)]
data.describe()

,barrierNumber,claimAmount,finishingPosition,num_runners,odds_perc,odds_scale,odds_win,prediction,probability,rank_win,runnerNumber
count,24671.000000,24671.000000,24671.000000,24671.000000,24671.000000,24671.000000,24671.000000,24671.000000,24671.000000,24671.000000,24671.000000
mean,5.866240,-0.359065,0.827409,10.469418,0.130312,0.098334,21.604201,0.107119,0.098334,5.924405,6.325970
std,3.601719,1.275640,1.280915,2.963039,0.136729,0.095735,29.628593,0.147186,0.122148,3.647433,3.887872
min,0.000000,-1.000000,0.000000,3.000000,0.003322,0.000778,1.010000,0.000000,0.000000,1.000000,1.000000
25%,3.000000,-1.000000,0.000000,8.000000,0.043478,0.031616,5.500000,0.018493,0.017527,3.000000,3.000000
50%,5.000000,-1.000000,0.000000,10.000000,0.090909,0.067818,11.000000,0.056451,0.055337,5.000000,6.000000
75%,8.000000,0.000000,2.000000,12.000000,0.181818,0.134127,23.000000,0.136720,0.133818,8.000000,9.000000
max,24.000000,5.000000,4.000000,22.000000,0.990099,0.893369,301.000000,1.000000,1.000000,23.000000,24.000000


In [7]:
# get label data
Y = data['finishingPosition'] == 1
Y = Y.astype(int)
Y.describe()

count    24671.000000
mean         0.098577
std          0.298100
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: finishingPosition, dtype: float64

In [9]:
# xp = fixed odds perc
xp = data['odds_perc']
xp.describe()

count    24671.000000
mean         0.130312
std          0.136729
min          0.003322
25%          0.043478
50%          0.090909
75%          0.181818
max          0.990099
Name: odds_perc, dtype: float64

In [10]:
# xs = odds scaled
xs = data['odds_scale']
xs.describe()

count    24671.000000
mean         0.098334
std          0.095735
min          0.000778
25%          0.031616
50%          0.067818
75%          0.134127
max          0.893369
Name: odds_scale, dtype: float64

In [11]:
# xr = win rank
xr = data['rank_win']
xr.describe()

count    24671.000000
mean         5.924405
std          3.647433
min          1.000000
25%          3.000000
50%          5.000000
75%          8.000000
max         23.000000
Name: rank_win, dtype: float64

In [12]:
# xn = get num runners
xn = data['num_runners']
xn.describe()

count    24671.000000
mean        10.469418
std          2.963039
min          3.000000
25%          8.000000
50%         10.000000
75%         12.000000
max         22.000000
Name: num_runners, dtype: float64

In [13]:
X = pd.concat([xp, xs, xr, xn], axis=1)
X.describe()
# sns.pairplot(pd.concat([X, Y], axis=1))

,odds_perc,odds_scale,rank_win,num_runners
count,24671.000000,24671.000000,24671.000000,24671.000000
mean,0.130312,0.098334,5.924405,10.469418
std,0.136729,0.095735,3.647433,2.963039
min,0.003322,0.000778,1.000000,3.000000
25%,0.043478,0.031616,3.000000,8.000000
50%,0.090909,0.067818,5.000000,10.000000
75%,0.181818,0.134127,8.000000,12.000000
max,0.990099,0.893369,23.000000,22.000000


In [14]:
# For a single-input model with 2 classes (binary classification):

n = len(X.columns)
print('input dimension = {}'.format(n))

epochs = 1000
print('epochs = {}'.format(epochs))

tag = 'R30x30'
file_name = '/Users/jaco/code/tabby/models/{}.h5'.format(tag)

try:
    model = load_model(file_name)
except OSError:
    model = Sequential()
    model.add(Dense(30, activation='relu', input_dim=n))
    model.add(Dense(30, activation='relu'))
#     model.add(Dense(40, activation='relu'))
#     model.add(Dense(40, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

input dimension = 4
epochs = 1000


In [15]:
# Train the model, iterating on the data in batches of 32 samples
tqdm = TQDMNotebookCallback()

tbCallBack = keras.callbacks.TensorBoard(
    log_dir='/Users/jaco/code/tabby/summary/{}'.format(tag), 
    histogram_freq=0,
    write_graph=True,
    write_images=True)

model.fit(
    X.as_matrix(), 
    Y.as_matrix(),
    validation_split=0.2,
    shuffle=True,
    epochs=epochs,
    batch_size=32,
    verbose=0,
    callbacks=[tqdm, tbCallBack])

# creates a HDF5 file 'my_model.h5'
model.save(file_name)